In this notebook we are going to use two packages to run `boruta` algorithm. One of them really interesting because incorpore the shap values in boruta process

In [ ]:
#!pip install boruta
#!pip install BorutaShap

In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_columns = None

In [2]:
df = pd.read_parquet("../data/data_processed.parquet")

print(df.shape)
df.head()

(116918, 70)


,tcuartosvi,thogar,tcuartos,tdormir,tsanitar,cuandi,cuanhoras,tpersona,buscando,puntaje,tiene_paredes_ladrillo,tiene_paredes_tapia,tiene_paredes_tabla,tiene_piso_cemento,tiene_piso_baldosa,tiene_piso_madera_tierra,es_estrato_1y2,tiene_inodoro_alcantarillado,tiene_letrina,tiene_agua_acueducto,cocinan_electricidad,cocinan_gas_natural,cocinan_gas_propano,tiene_eps_subsidiada,tiene_eps_contributiva,tiene_eps_ninguna,tiene_nivel_educativo_ninguna,tiene_nivel_educativo_primaria,tiene_nivel_educativo_secundaria,es_zona_urbana,tiene_servicio_telefono,es_vivi_casa_apto,tiene_energia,tiene_alcanta,tiene_gas,tiene_servicio_basura,tiene_acueducto,es_vivi_arriendo,es_vivi_prop_pagando,es_vivi_prop,es_vivi_otro,tiene_sanitario_dentro_vivi,tiene_sanitario_exclusivo,tiene_ducha,tiene_agua_7dias,tiene_agua_24h,tiene_espacio_cocinar,tiene_cocina_exclusiva_vivi,tiene_telefono_exclusivo_vivi,tiene_nevera,tiene_lavadora,tiene_televisor,tiene_televisor_parabolica,tiene_calentador,tiene_horno,tiene_aire,tiene_computador,tiene_equipo,tiene_moto,tiene_tractor,tiene_auto,tiene_bieraices,sexo_masculino,embarazada_o_con_hijos,asiste_centro_educativo,tiene_ingresos,es_extranjero,paga_arriendo,trabaja_servicios_domestico,ingresos
formulario,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
296071,4.0,1.0,4.0,2.0,1.0,0.0,0.0,2.0,0.0,38.89,0,1,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,500000.0
2000000,4.0,1.0,4.0,2.0,1.0,0.0,0.0,3.0,0.0,57.27,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,0,1,1,1,0,0,0,1280000.0
2000001,4.0,1.0,4.0,3.0,2.0,0.0,0.0,4.0,0.0,68.32,1,0,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,0,0,0,800000.0
2000002,2.0,1.0,2.0,2.0,1.0,0.0,0.0,5.0,0.0,36.30,1,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1057000.0
2000003,6.0,1.0,6.0,5.0,1.0,0.0,0.0,8.0,0.0,52.05,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,1294000.0


In this case, I'm going to omit the train-test split, the idea is just to ilustrate the Boruta algorithm.

In [3]:
y_name = "puntaje"
X = df.drop(columns=[y_name])
y = df[y_name].copy()

In [4]:
n_iter = 20 #n iterations (number of trials to perform)
alpha = 0.1 #"significance level"
seed = 421

## boruta.BorutaPy

In [5]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

In [8]:
# model to use
rf = RandomForestRegressor(
        n_jobs = -1,
        max_depth = 4,
        random_state = seed
)

In [21]:
# init boruta
boruta = BorutaPy(
       estimator = rf, 
       n_estimators = 100,
       max_iter = n_iter,
       alpha = alpha, 
       verbose = 1,
       random_state = seed
)
# fit Boruta (it accepts np.array, not pd.DataFrame)
boruta.fit(X.values, y.values)

Iteration: 1 / 20
Iteration: 2 / 20
Iteration: 3 / 20
Iteration: 4 / 20
Iteration: 5 / 20
Iteration: 6 / 20
Iteration: 7 / 20
Iteration: 8 / 20
Iteration: 9 / 20
Iteration: 10 / 20
Iteration: 11 / 20
Iteration: 12 / 20
Iteration: 13 / 20
Iteration: 14 / 20
Iteration: 15 / 20
Iteration: 16 / 20
Iteration: 17 / 20
Iteration: 18 / 20


BorutaPy finished running.

Iteration: 	19 / 20
Confirmed: 	15
Tentative: 	0
Rejected: 	54


BorutaPy(alpha=0.1,
         estimator=RandomForestRegressor(max_depth=4, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x7FF4B6411A40),
         max_iter=20, n_estimators=100,
         random_state=RandomState(MT19937) at 0x7FF4B6411A40, verbose=1)

In [22]:
# print results
confirmed_area = X.columns[boruta.support_].to_list()
tentative_area = X.columns[boruta.support_weak_].to_list()

In [23]:
confirmed_area

['thogar',
 'tiene_paredes_ladrillo',
 'tiene_inodoro_alcantarillado',
 'tiene_agua_acueducto',
 'tiene_eps_subsidiada',
 'tiene_nivel_educativo_ninguna',
 'tiene_nivel_educativo_primaria',
 'es_zona_urbana',
 'es_vivi_casa_apto',
 'tiene_alcanta',
 'tiene_sanitario_exclusivo',
 'tiene_ducha',
 'tiene_agua_7dias',
 'tiene_agua_24h',
 'tiene_lavadora']

## BorutaShap

In [9]:
from BorutaShap import BorutaShap

start = datetime.now()

# Creates a BorutaShap selector for regression
selector = BorutaShap(model = rf,
                      importance_measure = 'Shap',
                      classification = False, 
                      pvalue = alpha)
# Fits the selector
selector.fit(X = X, 
             y = y,
             n_trials = n_iter, 
             random_state = seed,
             sample = True,
             verbose = True)
# n_trials -> number of iterations for Boruta algorithm
# sample -> samples the data so it goes faster

finish = datetime.now()
print("Time execution:", finish - start)

  0%|          | 0/20 [00:00<?, ?it/s]

18 attributes confirmed important: ['tiene_alcanta', 'tiene_agua_7dias', 'es_vivi_casa_apto', 'tiene_paredes_tabla', 'es_zona_urbana', 'tiene_ducha', 'tiene_agua_24h', 'tiene_inodoro_alcantarillado', 'tiene_eps_subsidiada', 'tiene_lavadora', 'thogar', 'tiene_paredes_ladrillo', 'tiene_sanitario_exclusivo', 'tiene_agua_acueducto', 'tiene_nivel_educativo_ninguna', 'tiene_nivel_educativo_primaria', 'tcuartosvi', 'tiene_sanitario_dentro_vivi']
51 attributes confirmed unimportant: ['es_extranjero', 'tiene_nevera', 'tiene_horno', 'sexo_masculino', 'cocinan_gas_propano', 'tcuartos', 'cuandi', 'tiene_gas', 'tiene_moto', 'tiene_servicio_basura', 'tiene_eps_contributiva', 'tiene_tractor', 'tiene_acueducto', 'tiene_piso_cemento', 'tiene_paredes_tapia', 'cocinan_gas_natural', 'es_estrato_1y2', 'tiene_nivel_educativo_secundaria', 'tsanitar', 'paga_arriendo', 'tiene_piso_baldosa', 'es_vivi_arriendo', 'ingresos', 'asiste_centro_educativo', 'cuanhoras', 'tiene_energia', 'tdormir', 'es_vivi_prop', 'tien

In [32]:
reject_area = selector.rejected
confirmed_area = selector.accepted
tentative_area = selector.tentative

In [33]:
confirmed_area

['tiene_alcanta',
 'tiene_agua_7dias',
 'es_vivi_casa_apto',
 'tiene_paredes_tabla',
 'es_zona_urbana',
 'tiene_ducha',
 'tiene_agua_24h',
 'tiene_inodoro_alcantarillado',
 'tiene_eps_subsidiada',
 'tiene_lavadora',
 'thogar',
 'tiene_paredes_ladrillo',
 'tiene_sanitario_exclusivo',
 'tiene_agua_acueducto',
 'tiene_nivel_educativo_ninguna',
 'tiene_nivel_educativo_primaria',
 'tcuartosvi',
 'tiene_sanitario_dentro_vivi']

---